# 16. Interactive Explorer

VAMOS provides a built-in interactive dashboard to explore Pareto fronts, inspecting decision variables and objective trade-offs.

This feature requires `plotly` and `ipywidgets`.

In [ ]:
import io
from vamos.ux.api import explore_result_front

try:
    import plotly.graph_objects as go
    import ipywidgets as widgets
    from IPython.display import display, clear_output

    HAS_REQ = True
    print("Interactive libraries loaded.")
except ImportError:
    HAS_REQ = False
    print("This notebook requires 'plotly' and 'ipywidgets'. Please install them.")

## 1. Quick Example

Run a simple optimization and explore the results.

In [ ]:
from vamos import optimize

if HAS_REQ:
    print("Running ZDT1... (approx 1-2 sec)")
    result = optimize("zdt1", algorithm="nsgaii", pop_size=100, max_evaluations=5000, seed=42, verbose=False)
    print("Launching explorer...")
    # explore_result_front(result) launches an interactive Plotly figure
    # You can zoom, pan, and hover over points to see variable values.
    explore_result_front(result, title="ZDT1 Pareto Explorer")

## 2. Interactive Experiment Runner

Use widgets to configure and run experiments on the fly, then visualize using `explore_result_front(result)`.

In [ ]:
if HAS_REQ:
    # Setup Widgets
    prob_select = widgets.Dropdown(options=["zdt1", "zdt2", "zdt3", "dtlz2"], value="zdt2", description="Problem:")

    algo_select = widgets.Dropdown(options=["nsgaii", "moead", "spea2"], value="nsgaii", description="Algorithm:")

    evals_slider = widgets.IntSlider(value=5000, min=1000, max=20000, step=1000, description="Budget:")

    run_btn = widgets.Button(description="Run Experiment", button_style="primary")
    out_area = widgets.Output()

    def on_run(_):
        with out_area:
            clear_output()
            print(f"Optimization Started: {prob_select.value} with {algo_select.value}...")

            try:
                res = optimize(problem=prob_select.value, algorithm=algo_select.value, max_evaluations=evals_slider.value, pop_size=100, seed=42)
                print("Optimization Complete. Rendering Plot...")
                explore_result_front(res, title=f"{prob_select.value.upper()} Results")
            except Exception as e:
                print(f"Error: {e}")

    run_btn.on_click(on_run)

    # Display Dashboard
    ui = widgets.VBox([widgets.HBox([prob_select, algo_select]), widgets.HBox([evals_slider, run_btn]), out_area])

    display(ui)